# Biomedical RAG QA (Offline Corpus + Gradio UI)
This notebook loads a static biomedical abstract corpus, builds embeddings with BioBERT, and answers questions using BioBART + Gradio.

In [ ]:
# Install dependencies
!pip install sentence-transformers faiss-cpu transformers gradio accelerate

In [ ]:
# Sample biomedical corpus
corpus = [
    'The hippocampus plays a key role in memory formation and spatial navigation.',
    'Mutations in the CFTR gene cause cystic fibrosis, affecting the lungs and digestive system.',
    'Insulin regulates glucose metabolism and is produced by the beta cells of the pancreas.',
    'The BRCA1 and BRCA2 genes are involved in DNA repair and are linked to breast cancer.',
    'Hypertension is a condition characterized by persistently elevated blood pressure.',
    'The immune system defends against infectious agents through innate and adaptive mechanisms.',
    'Alzheimer’s disease is associated with amyloid plaques and neurofibrillary tangles.',
    'MRI is a medical imaging technique used to visualize detailed internal structures.',
    'The liver performs vital metabolic and detoxification functions.',
    'Vaccines work by stimulating an immune response to specific antigens.'
] * 100  # replicate to simulate 1000 entries

In [ ]:
# Generate embeddings
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer('pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb')
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=False, show_progress_bar=True)

index = faiss.IndexFlatL2(len(corpus_embeddings[0]))
index.add(np.array(corpus_embeddings))

In [ ]:
# Load BioBART model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = 'bionlp/biobart-v2.0'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
qa_pipeline = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

In [ ]:
# Define RAG QA function
def rag_qa(query):
    query_embedding = embedder.encode([query])[0]
    D, I = index.search(np.array([query_embedding]), k=5)
    retrieved_docs = [corpus[i] for i in I[0]]
    context = ' '.join(retrieved_docs)
    prompt = f'question: {query} context: {context}'
    result = qa_pipeline(prompt, max_length=256)[0]['generated_text']
    return result

In [ ]:
# Gradio UI
import gradio as gr
demo = gr.Interface(
    fn=rag_qa,
    inputs=gr.Textbox(label='Ask a biomedical question'),
    outputs=gr.Textbox(label='AI-generated answer'),
    title='Biomedical RAG QA (BioBERT + BioBART)',
    description='Open-source retrieval augmented generation demo using biomedical knowledge.'
)
demo.launch(debug=True)